# **Credit Card Fraud Detection using Cost-Sensitive Learning.**

> [**Kaggle Dataset**](https://www.kaggle.com/mlg-ulb/creditcardfraud)

In [ ]:
# Install Kaggle.
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
# Files Upload.
from google.colab import files

files.upload()

In [ ]:
# Create a Kaggle Folder.
!mkdir ~/.kaggle

# Copy the kaggle.json to the folder created.
!cp kaggle.json ~/.kaggle/

# Permission for the json file to act.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Dataset Download.
!kaggle datasets download -d mlg-ulb/creditcardfraud

In [ ]:
# Unzip Dataset.
!unzip creditcardfraud.zip

In [ ]:
!pip install tensorflow_addons

In [ ]:
# Import Library.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras import initializers, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.metrics import CohenKappa
import warnings

warnings.filterwarnings("ignore")

# Import Dataset.
data = pd.read_csv("creditcard.csv")
data.head()

In [ ]:
# Class Distribution of Dataset.
print(data["Class"].value_counts())

In [ ]:
# Visualize the Class Distribution.
pd.value_counts(data["Class"]).plot.bar()

In [ ]:
# Split Dataset into Feature and Target Set.
X = data.iloc[:, 1:30]
y = data.iloc[:, -1]

# Split Dataset into Training and Test Set.
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Model Configuration.
BATCH_SIZE = 256
NO_EPOCHS = 100
NO_CLASSES = 2
VALIDATION_SPLIT = 0.2
VERBOSITY = 1
my_callbacks = [
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True)
]


# Model Architecture/Pipeline.
def create_model():
    model = Sequential()

    model.add(
        Dense(
            units=100,
            activation="relu",
            kernel_initializer=tf.keras.initializers.GlorotNormal(),
            kernel_regularizer=tf.keras.regularizers.l2(l2=0.01),
        )
    )
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(
        Dense(
            units=100,
            activation="relu",
            kernel_initializer=tf.keras.initializers.GlorotNormal(),
            kernel_regularizer=tf.keras.regularizers.l2(l2=0.01),
        )
    )
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(
        Dense(
            units=100,
            activation="relu",
            kernel_initializer=tf.keras.initializers.GlorotNormal(),
            kernel_regularizer=tf.keras.regularizers.l2(l2=0.01),
        )
    )
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(
        Dense(
            units=100,
            activation="relu",
            kernel_initializer=tf.keras.initializers.GlorotNormal(),
            kernel_regularizer=tf.keras.regularizers.l2(l2=0.01),
        )
    )
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(
        Dense(
            units=100,
            activation="relu",
            kernel_initializer=tf.keras.initializers.GlorotNormal(),
            kernel_regularizer=tf.keras.regularizers.l2(l2=0.01),
        )
    )
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Dense(1, activation="sigmoid"))

    # Compile the Model.
    model.compile(
        optimizer="adam",
        loss="binary_crossentropy",
        metrics=["accuracy", CohenKappa(num_classes=NO_CLASSES)],
    )
    return model

In [ ]:
# Call the Model Architecture.
model = create_model()

# Build the Model.
model.build(X_train.shape)
model.summary()

In [ ]:
"""
Weighted Neural Network:
    1. Define Weights.
    2. Fit the model with those specific weights.
"""

# The class distribution of the "creditcard.csv" dataset has a 1:560 ratio for the minority class to the majority class.
weights_assigned = {0: 1, 1: 560}

In [ ]:
# Fit the Model.
history = model.fit(
    X_train,
    y_train,
    class_weight=weights_assigned,
    batch_size=BATCH_SIZE,
    epochs=NO_EPOCHS,
    verbose=VERBOSITY,
    validation_split=VALIDATION_SPLIT,
    callbacks=my_callbacks,
)

In [ ]:
# Plot Training and Validation Graph.
kap = history.history["cohen_kappa"]
val_kappa = history.history["val_cohen_kappa"]
epochs = len(kap)
plt.plot(np.arange(epochs), kap, label="Training Kappa")
plt.plot(np.arange(epochs), val_kappa, "g", label="Validation Kappa")
plt.legend()
plt.show()

In [ ]:
# Model Evaluation.
print(model.evaluate(X_test, y_test))

In [ ]:
# Performance Score.
y_pred = model.predict(X_test)
print("\n ROC-AUC Score is ", roc_auc_score(y_test, y_pred))

2226/2226 [==============================] - 4s 2ms/step

 ROC-AUC Score is  0.9507912700365344
